In [1]:
import numpy as np
import pandas as pd
import cv2

In [9]:
def extract(method, image_path):
    image = cv2.imread(image_path)
    # apple color boundaries [B, G, R]
    lower = [0, 0, 0]
    upper = [179, 255, 125]

    # create NumPy arrays from the boundaries
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    if method == 'hsv':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # Find the colors within the specified boundaries and apply the mask
        binary = cv2.inRange(image, lower, upper)
        cv2.imwrite('z_binary.jpg', binary)
        # _, thresh = cv2.threshold(binary, 40, 255, 0)

        # # cv2.imwrite('z_binary.jpg', thresh)
        
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
        largest_item = sorted_contours[0]
        area = cv2.contourArea(largest_item)
        
        # Obtain binary mask from largest contour
        mask = np.zeros_like(image[:,:,0])
        cv2.drawContours(mask, [largest_item], -1, 255, -1)

    if method == 'canny':
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        cv2.imwrite('z_gray.jpg', gray)
        v = np.median(gray)
        sigma = 0.33
        
        # Applying automatic Canny edge detection using the computed median
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        edge = cv2.Canny(gray, lower, upper)
        cv2.imwrite('z_edge_0.jpg', edge)
        
        # Close the edges
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
        edge = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)
        cv2.imwrite('z_edge.jpg', edge)
        
        contours, _ = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [largest_contour], -1, 255, -1)

    if method == 'otsu':
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        cv2.imwrite('z_gray.jpg', gray)
        
        # Ostu thresholding
        ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        cv2.imwrite('z_otsu.jpg', thresh)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [largest_contour], -1, 255, -1)
    cv2.imwrite('z_mask.jpg', mask)
    return mask
            

In [3]:
dataset = pd.read_csv('./assets/v2/dataset_v2.csv')
han = dataset.sample(5, random_state=44)
print(han.iloc[4]['image_path'])

/mnt/data/dataset/apple-weight-dataset/v2/3145-87/image0005362.png


In [15]:
methods = {1: 'hsv', 2: 'canny', 3: 'otsu'}
image_path = han.iloc[4]['image_path']
extract(methods[1], image_path)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)